In [23]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
import os
import nest_asyncio

#Used by LlamaIndex
nest_asyncio.apply()

#API key for AzureOpenAI. Replace with your own key
api_key = "4e4ab31800a64ae892cbb768fe28c0fc"
#Azure Endpoint URL. Replace with your own URL
azure_endpoint = "https://agentic-ai-course-kumaran.openai.azure.com/"

#Note : Function calling support only available in GPT-4+
#Create the LLM object
Settings.llm=AzureOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-08-01-preview",
)

#Set up the embedding model for RAG
Settings.embed_model= AzureOpenAIEmbedding(
    model="text-embedding-3-large",
    deployment_name="text-embedding-3-large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version="2023-05-15",
)

### Set up wikipedia tools

In [16]:
from llama_index.tools.wikipedia import WikipediaToolSpec
from llama_index.core.tools.tool_spec.load_and_search import (
    LoadAndSearchToolSpec,
)

# Get list of wikipedia tools
wiki_spec = WikipediaToolSpec()
tool = wiki_spec.to_tool_list()[1]
wiki_tools = LoadAndSearchToolSpec.from_defaults(tool).to_tool_list()

#Review the list of tools
for tool in wiki_tools:
    print( "--------\n",tool.metadata.name, tool.metadata.description)

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    


### Set up RAG for medications

In [48]:
#from llama_index.core import SimpleDirectoryReader
#from llama_index.readers.file import CSVReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.readers.json import JSONReader

medication_docs = JSONReader().load_data(
        input_file="datasets/Medications and dosages.json"
)
#Split the document
splitter=SentenceSplitter(chunk_size=200)
medication_nodes=splitter.get_nodes_from_documents(medication_docs)

#Index the document in memory
medication_index=VectorStoreIndex(medication_nodes)

#Create a query engine tool based on the CSV file
medication_query_engine = medication_index.as_query_engine()
medication_tool = QueryEngineTool.from_defaults(
    query_engine=medication_query_engine,
    description=(
        "Provides list of popular medications, diseases they are used to treat for and recommended dosages"
    ),
)

In [49]:
from llama_index.core.agent.react.base import ReActAgent
#Consolidate list of tools
healthcare_tools =wiki_tools + [medication_tool]

context = """
You are an healthcare assistant who can answer questions regarding 
diseases, their symptoms and medications for them.

Use the wikipedia tools to answer questions about symptoms and possible 
diseases associated with those symptoms.

Use the medication tool to find list of medications and dosages for a given
disease.

Use only the tools provided to answer questions and NOT your own memory.

"""

#Create a pre-built ReAct agent
healthcare_agent = ReActAgent.from_tools(
                healthcare_tools,
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


In [56]:
response=healthcare_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 5af86dac-8dd8-40b0-af7e-3539d6b208f0. Step input: Which medication should i take for arthritis?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'medication for arthritis'}
Observation: Naproxen is used for pain and inflammation, which can be associated with arthritis.
> Running step bdc514ef-781e-4b99-ae30-a2c548738587. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Naproxen is a medication commonly used to manage pain and inflammation associated with arthritis. However, it's important to consult with a healthcare professional to determine the most appropriate medication for your specific condition and health needs.
*******
 Response :  Naproxen is a medication commonly used to manage pain and inflammation associated with arthritis. However, it's important to consult with a healthcare professio

In [57]:
response=healthcare_agent.chat("I have fever and soar throat, what is the possible cause?")
print("*******\n Response : ",response.response)

> Running step fecc8dc6-b28c-43e3-a9ce-7dacb132ea83. Step input: I have fever and soar throat, what is the possible cause?
Thought: The current language of the user is English. I need to use a tool to help me answer the question about possible diseases associated with the symptoms of fever and sore throat.
Action: search_data
Action Input: {'query': 'fever and sore throat causes'}
Observation: Content loaded! You can now search the information using read_search_data
> Running step a886cf0b-6e49-4db2-9533-d077aa092a87. Step input: None
Thought: I will now use the read_search_data tool to extract information about the possible causes of fever and sore throat.
Action: read_search_data
Action Input: {'query': 'What are the possible causes of fever and sore throat?'}
Observation: Possible causes of fever and sore throat include viral infections, group A streptococcal (GAS) bacterial infection, pharyngitis, tonsillitis, and gastroesophageal reflux disease.
> Running step 9c571a31-449d-4a83-b